In [5]:
import pysrt
subs = pysrt.open("C:\\Users\\AI_Dep\\Downloads\\Money Heist Season (Lacasa De Papel) 1 Subtitles _ AllNaijaEntertainment.com\\Season 01 - Downloaded From AllNaijaEntertainment.com\\La.casa.de.papel.S01E01.WEBRip.Netflix.srt")

In [6]:
len(subs)

597

In [7]:
subs[0].start.milliseconds

800

In [8]:
subs[0].end

SubRipTime(0, 0, 8, 760)

In [9]:
import pandas as pd

In [10]:

data = {}
texts = []
hours = []
minutes = []
seconds = []
end_hours = []
end_minutes = []
end_seconds = []
end_milliseconds = []
milliseconds = []
for i in range(len(subs)):
    texts.append(subs[i].text)
    hours.append(subs[i].start.hours)
    minutes.append(subs[i].start.minutes)
    seconds.append(subs[i].start.seconds)
    milliseconds.append(subs[i].start.milliseconds)
    end_hours.append(subs[i].end.hours)
    end_minutes.append(subs[i].end.minutes)
    end_seconds.append(subs[i].end.seconds)
    end_milliseconds.append(subs[i].end.milliseconds)

In [11]:
data['text'] = texts
data['hour'] = hours
data['minutes'] = minutes
data['seconds'] = seconds
data['milliseconds'] = milliseconds
data['end_hour'] = end_hours
data['end_minutes'] = end_minutes
data['end_seconds'] = end_seconds
data['end_milliseconds'] = end_milliseconds

In [12]:
df = pd.DataFrame(data)


In [13]:
df

,text,hour,minutes,seconds,milliseconds,end_hour,end_minutes,end_seconds,end_milliseconds
0,Stop or I’ll shoot!,0,0,6,800,0,0,8,760
1,No!,0,0,8,880,0,0,9,720
2,"""My name’s Tokyo.",0,0,25,200,0,0,26,400
3,"But when this story started,\nthat wasn’t my n...",0,0,28,560,0,0,31,800
4,This was me.,0,0,33,840,0,0,34,600
...,...,...,...,...,...,...,...,...,...
592,"So if you’re shot in the heart,",0,46,24,920,0,46,26,760
593,you won’t even hear\nthe bullet that has kille...,0,46,27,680,0,46,29,880
594,"That’s how I knew I had fucked it all up,",0,46,31,280,0,46,33,360
595,in a millisecond,0,46,34,0,0,46,35,600


3

In [11]:
from pydub import AudioSegment

audio_file= "D:\\wav-Casa del Papel\\La.casa.de.papel.A.K.A.Money.Heist.S01E01.DUAL-AUDIO.SPA-ENG.720p.10bit.WEBRip.2CH.x265.HEVC-PSA.wav"
audio = AudioSegment.from_wav(audio_file)

#start = df.iloc[0]['seconds'] * 1000
start = 0
end = df.iloc[1]['seconds'] * 1000

audio_chunk=audio[start:end]
audio_chunk.export( "audio_chunk_{}.wav".format(end), format="wav")




<_io.BufferedRandom name='audio_chunk_8000.wav'>

In [29]:
# Import the AudioSegment class for processing audio and the 
# split_on_silence function for separating out silent chunks.
from pydub import AudioSegment
from pydub.silence import split_on_silence

# Define a function to normalize a chunk to a target amplitude.
def match_target_amplitude(aChunk, target_dBFS):
    ''' Normalize given audio chunk '''
    change_in_dBFS = target_dBFS - aChunk.dBFS
    return aChunk.apply_gain(change_in_dBFS)

# Load your audio.
song = AudioSegment.from_wav("C:\\Users\\AI_Dep\\Work\\Untitled Folder 2\\audio_chunk_8000.wav")

# Split track where the silence is 2 seconds or more and get chunks using 
# the imported function.
chunks = split_on_silence (
    # Use the loaded audio.
    song, 
    # Specify that a silent chunk must be at least 2 seconds or 2000 ms long.
    min_silence_len = 2000,
    # Consider a chunk silent if it's quieter than -16 dBFS.
    # (You may want to adjust this parameter.)
    silence_thresh = -100
)



In [30]:
len(chunks)

1

In [31]:
# Process each chunk with your parameters
for i, chunk in enumerate(chunks):
    # Create a silence chunk that's 0.5 seconds (or 500 ms) long for padding.
    silence_chunk = AudioSegment.silent(duration=500)

    # Add the padding chunk to beginning and end of the entire chunk.
    audio_chunk = silence_chunk + chunk + silence_chunk

    # Normalize the entire chunk.
    normalized_chunk = match_target_amplitude(audio_chunk, -20.0)

    # Export the audio chunk with new bitrate.
    print("Exporting chunk{0}.mp3.".format(i))
    normalized_chunk.export(
        ".//chunk{0}.wav".format(i),
        bitrate = "192k",
        format = "wav"
    )

Exporting chunk0.mp3.


In [1]:
import argparse
import os
from pathlib import Path
from encoder.audio import trim_long_silences

import librosa
import numpy as np
import soundfile as sf
import torch

from encoder import inference as encoder
from encoder.params_model import model_embedding_size as speaker_embedding_size
from synthesizer.inference import Synthesizer
from utils.argutils import print_args
from utils.default_models import ensure_default_models
from vocoder import inference as vocoder


def voice_change(text, file_path, encoder_path = 'saved_models\\default\\encoder.pt', synth_path = 'saved_models\\default\\synthesizer.pt', vocoder_path = 'saved_models\\default\\vocoder.pt', trim_silence = True, seed = None, cpu=False, no_sound=False):

    if cpu:
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


    if torch.cuda.is_available():
        device_id = torch.cuda.current_device()
        gpu_properties = torch.cuda.get_device_properties(device_id)
        print("Found %d GPUs available. Using GPU %d (%s) of compute capability %d.%d with "
            "%.1fGb total memory.\n" %
            (torch.cuda.device_count(),
            device_id,
            gpu_properties.name,
            gpu_properties.major,
            gpu_properties.minor,
            gpu_properties.total_memory / 1e9))
    else:
        print("Using CPU for inference.\n")

    ensure_default_models(Path("saved_models"))
    encoder.load_model(encoder_path)
    synthesizer = Synthesizer(synth_path)
    vocoder.load_model(vocoder_path, verbose=False)

    encoder.embed_utterance(np.zeros(encoder.sampling_rate))
    embed = np.random.rand(speaker_embedding_size)
    embed /= np.linalg.norm(embed)
    embeds = [embed, np.zeros(speaker_embedding_size)]
    texts = ["test 1", "test 2"]
    mels = synthesizer.synthesize_spectrograms(texts, embeds)
    mel = np.concatenate(mels, axis=1)
    no_action = lambda *args: None

    vocoder.infer_waveform(mel, target=200, overlap=50, progress_callback=no_action)

    num_generated = 0
    a = True
    while a:
        try:
            # Get the reference audio filepath
            message = file_path
            in_fpath = Path(message.replace("\"", "").replace("\'", ""))

            preprocessed_wav = encoder.preprocess_wav(in_fpath)
            original_wav, sampling_rate = librosa.load(str(in_fpath))
            preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
            embed = encoder.embed_utterance(preprocessed_wav)


            
            # text = input("Write a sentence (+-20 words) to be synthesized:\n")
            if seed is not None:
                torch.t(torch.tensor([seed]))
                synthesizer = Synthesizer(synth_path)

            texts = [text]
            embeds = [embed]
            specs = synthesizer.synthesize_spectrograms(texts, embeds)
            spec = specs[0]

            if seed is not None:
                torch.manual_seed(seed)
                vocoder.load_model(vocoder_path, verbose=False)
            generated_wav = vocoder.infer_waveform(spec)
            generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
            generated_wav = encoder.preprocess_wav(generated_wav)
            # if not no_sound:
            #     import sounddevice as sd
            #     try:
            #         sd.stop()
            #         sd.play(generated_wav, synthesizer.sample_rate)
            #     except sd.PortAudioError as e:
            #         print("\nCaught exception: %s" % repr(e))
            #         print("Continuing without audio playback. Suppress this message with the \"--no_sound\" flag.\n")
            #     except:
            #         raise
            # print('cv')
            print(file_path)
            filename = f"{file_path}_generated.wav"
            # print('filename d')
            # if trim_silence:
            #    trim_long_silences(generated_wav)
            # sf.write(filename, generated_wav.astype(np.float32), synthesizer.sample_rate)
            # print('write d')
            num_generated += 1
            # print("\nSaved output as %s\n\n" % filename)


        except Exception as e:
            print("Caught exception: %s" % repr(e))
            print("Restarting\n")
            
        finally:
            a = False
            return generated_wav

if __name__=="__main__":
    voice_change("You should suck some goat dick, it's very tasty.", 'D:\\Trim\\Casa Del Papel1.wav', seed= 0)

Found 1 GPUs available. Using GPU 0 (NVIDIA GeForce RTX 3080) of compute capability 8.6 with 10.7Gb total memory.

Synthesizer using device: cuda
Trainable Parameters: 4.481M
Trainable Parameters: 30.870M

| Generating 1/1


Done.

Synthesizer using device: cuda
Trainable Parameters: 30.870M

| Generating 1/1


Done.

Trainable Parameters: 4.481M
{| ███░░░░░░░░░░░░░ 6000/38400 | Batch Size: 4 | Gen Rate: 4.0kHz | }

UnboundLocalError: local variable 'generated_wav' referenced before assignment

# Totul pus cap la cap

In [25]:
import pysrt
import pandas as pd
from pydub import AudioSegment
from pydub.silence import split_on_silence
import argparse
import os
from pathlib import Path
from encoder.audio import trim_long_silences
from scipy.io.wavfile import write
import librosa
import numpy as np
import soundfile as sf
import torch

from encoder import inference as encoder
from encoder.params_model import model_embedding_size as speaker_embedding_size
from synthesizer.inference import Synthesizer
from utils.argutils import print_args
from utils.default_models import ensure_default_models
from vocoder import inference as vocoder





# Subtitles reading
subs = pysrt.open("C:\\Users\\AI_Dep\\Downloads\\Money Heist Season (Lacasa De Papel) 1 Subtitles _ AllNaijaEntertainment.com\\Season 01 - Downloaded From AllNaijaEntertainment.com\\La.casa.de.papel.S01E01.WEBRip.Netflix.srt")


# Create dataframe with all datas
data = {}
texts = []
hours = []
minutes = []
seconds = []
end_hours = []
end_minutes = []
end_seconds = []
end_milliseconds = []
milliseconds = []
for i in range(len(subs)):
    texts.append(subs[i].text)
    hours.append(subs[i].start.hours)
    minutes.append(subs[i].start.minutes)
    seconds.append(subs[i].start.seconds)
    milliseconds.append(subs[i].start.milliseconds)
    end_hours.append(subs[i].end.hours)
    end_minutes.append(subs[i].end.minutes)
    end_seconds.append(subs[i].end.seconds)
    end_milliseconds.append(subs[i].end.milliseconds)
    
    
data['text'] = texts
data['hour'] = hours
data['minutes'] = minutes
data['seconds'] = seconds
data['milliseconds'] = milliseconds
data['end_hour'] = end_hours
data['end_minutes'] = end_minutes
data['end_seconds'] = end_seconds
data['end_milliseconds'] = end_milliseconds

df = pd.DataFrame(data)



audio_file= "D:\\wav-Casa del Papel\\La.casa.de.papel.A.K.A.Money.Heist.S01E01.DUAL-AUDIO.SPA-ENG.720p.10bit.WEBRip.2CH.x265.HEVC-PSA.wav"
audio = AudioSegment.from_wav(audio_file)

i= 0
start = df.iloc[0]['minutes'] * 60000 +  df.iloc[0]['seconds'] * 1000 + df.iloc[0]['milliseconds']

end = df.iloc[0]['end_minutes'] * 60000 +  df.iloc[0]['end_seconds'] * 1000 + df.iloc[0]['end_milliseconds']

print(start)
print(end)

audio_chunk=audio[start:end]
audio_chunk.export( "segment1.wav", format="wav")


def voice_change(text, file_path, encoder_path = 'saved_models\\default\\encoder.pt', synth_path = 'saved_models\\default\\synthesizer.pt', vocoder_path = 'saved_models\\default\\vocoder.pt', trim_silence = True, seed = None, cpu=False, no_sound=False):

    if cpu:
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


    if torch.cuda.is_available():
        device_id = torch.cuda.current_device()
        gpu_properties = torch.cuda.get_device_properties(device_id)
        print("Found %d GPUs available. Using GPU %d (%s) of compute capability %d.%d with "
            "%.1fGb total memory.\n" %
            (torch.cuda.device_count(),
            device_id,
            gpu_properties.name,
            gpu_properties.major,
            gpu_properties.minor,
            gpu_properties.total_memory / 1e9))
    else:
        print("Using CPU for inference.\n")

    ensure_default_models(Path("saved_models"))
    encoder.load_model(encoder_path)
    synthesizer = Synthesizer(synth_path)
    vocoder.load_model(vocoder_path, verbose=False)

    encoder.embed_utterance(np.zeros(encoder.sampling_rate))
    embed = np.random.rand(speaker_embedding_size)
    embed /= np.linalg.norm(embed)
    embeds = [embed, np.zeros(speaker_embedding_size)]
    texts = ["test 1", "test 2"]
    mels = synthesizer.synthesize_spectrograms(texts, embeds)
    mel = np.concatenate(mels, axis=1)
    no_action = lambda *args: None

    vocoder.infer_waveform(mel, target=200, overlap=50, progress_callback=no_action)

    num_generated = 0
    a = True
    while a:
        try:
            # Get the reference audio filepath
            message = file_path
            in_fpath = Path(message.replace("\"", "").replace("\'", ""))

            preprocessed_wav = encoder.preprocess_wav(in_fpath)
            original_wav, sampling_rate = librosa.load(str(in_fpath))
            preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
            embed = encoder.embed_utterance(preprocessed_wav)
            # text = input("Write a sentence (+-20 words) to be synthesized:\n")
            if seed is not None:
                torch.t(torch.tensor([seed]))
                synthesizer = Synthesizer(synth_path)

            texts = [text]
            embeds = [embed]
            specs = synthesizer.synthesize_spectrograms(texts, embeds)
            spec = specs[0]

            if seed is not None:
                torch.manual_seed(seed)
                vocoder.load_model(vocoder_path, verbose=False)
            generated_wav = vocoder.infer_waveform(spec)
            generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
            generated_wav = encoder.preprocess_wav(generated_wav)
            # if not no_sound:
            #     import sounddevice as sd
            #     try:
            #         sd.stop()
            #         sd.play(generated_wav, synthesizer.sample_rate)
            #     except sd.PortAudioError as e:
            #         print("\nCaught exception: %s" % repr(e))
            #         print("Continuing without audio playback. Suppress this message with the \"--no_sound\" flag.\n")
            #     except:
            #         raise
            # print('cv')
            try:
                number = max([int(i.split('.')[0]) for i in os.listdir('outputs/')])+1
            except:
                number = 1
            filename = f"outputs/{number}.wav"
            sf.write(filename, generated_wav.astype(np.float32), synthesizer.sample_rate)

            # print('write d')
            num_generated += 1
            # print("\nSaved output as %s\n\n" % filename)


        except Exception as e:
            print("Caught exception: %s" % repr(e))
            print("Restarting\n")
            
        finally:
            a = False
            return generated_wav
        




voice_change(df.iloc[0]['text'], 'segment1.wav', seed= 0)

wav = os.listdir("C:\\Users\\AI_Dep\\Work\\Untitled Folder 2\\outputs")[-1]

to_add_wav  = AudioSegment.from_wav("C:\\Users\\AI_Dep\\Work\\Untitled Folder 2\\outputs\\"+wav)

try:
    current_movie = AudioSegment.from_wav("current_movie.wav")
except:
    current_movie = audio[:start]

combined_movie = current_movie + to_add_wav

combined_movie.export( "current_movie.wav", format="wav")

6800
8760
Found 1 GPUs available. Using GPU 0 (NVIDIA GeForce RTX 3080) of compute capability 8.6 with 10.7Gb total memory.

Synthesizer using device: cuda
Trainable Parameters: 4.481M
Trainable Parameters: 30.870M

| Generating 1/1


Done.

Synthesizer using device: cuda
Trainable Parameters: 30.870M

| Generating 1/1


Done.

Trainable Parameters: 4.481M
{| ████████████████ 38000/38400 | Batch Size: 4 | Gen Rate: 4.6kHz | }

<_io.BufferedRandom name='current_movie.wav'>

In [22]:
from pydub import AudioSegment

trump  = AudioSegment.from_wav("C:\\Users\\AI_Dep\\Work\\test_deepFake\\Real-Time-Voice-Cloning\\trump10.wav")
beep     = AudioSegment.from_wav("C:\\Users\\AI_Dep\\Work\\Untitled Folder 2\\outputs\\5.wav")

combined_sounds = trump + beep
combined_sounds.export("segment1trump.wav", format="wav")


<_io.BufferedRandom name='segment1trump.wav'>

In [37]:

from scipy.io.wavfile import write
write('test.wav',len(dap),data = dap)

In [24]:
wav

'6.wav'

In [4]:
df

,text,hour,minutes,seconds,milliseconds,end_hour,end_minutes,end_seconds,end_milliseconds
0,Stop or I’ll shoot!,0,0,6,800,0,0,6,800
1,No!,0,0,8,880,0,0,8,880
2,"""My name’s Tokyo.",0,0,25,200,0,0,25,200
3,"But when this story started,\nthat wasn’t my n...",0,0,28,560,0,0,28,560
4,This was me.,0,0,33,840,0,0,33,840
...,...,...,...,...,...,...,...,...,...
592,"So if you’re shot in the heart,",0,46,24,920,0,46,24,920
593,you won’t even hear\nthe bullet that has kille...,0,46,27,680,0,46,27,680
594,"That’s how I knew I had fucked it all up,",0,46,31,280,0,46,31,280
595,in a millisecond,0,46,34,0,0,46,34,0


In [31]:
import glob
import os

Dir = 'D:\\Train\\SV2TTS\\encoder\\'  #Source directory containing encoder preprocessed files 
OUTPUTFILE = r'D:\\Train\\SV2TTS\\folder.txt' #.txt destination directory

path = os.path.join(Dir, '*')
for folder in glob.glob(path):
    with open(OUTPUTFILE, 'a') as f:
        if len(os.listdir(folder)) < 2:
            f.write("{0} YEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEES'\n' ".format(folder))
        else:
            f.write("{0} is a directory with files'\n' ".format(folder))